!pip install --upgrade "ibm-watson>=7.0.1"

# Description

The goal of these notebook is to sucessfully download the actions and upload them witht he new Custom
extension ids in case we modified them for some reason. 



In [ ]:
import pprint
from dotenv import load_dotenv
import os
load_dotenv()

# Assistant setup 

In [ ]:
URL = 'https://api.us-east.assistant.watson.cloud.ibm.com/'
API_KEY = os.getenv(key='API_KEY')

# put here the name of the assistant to replace extensions ids 
#ASSISTANT_NAME = 'Experiments_Ask_Proc_Main'

CHANGE_LOG_DESCRIPTION = '''
----- Process Description -----
Copy all from Ask_Proc_Main to DEV_TEST

----- Changes -----



'''

ASSISTANT_FROM = 'Ask_Proc_Main'
ASSISTANT_TO = 'PREPROD_PROD_AskProc'

BACKUP_FOLDER = 'assistant_backups/'

ONLY_EPM_API_CHANGE = True

MODIFY = [
    {
        'id_from': "6c319007-41f4-4cce-8368-7a6d38905888", #  epm ask_proc_main
        'id_to': "8acb5354-2e54-401a-805a-1e9927167311" # epm_api_v21 ask_proc_main
        },
        {
        'id_from': "0710d532e446af11a39ab1c8d3d4b5281b94cfd91550eb8f2cf6d9ff669b3290", #   epm ask_proc_main
        'id_to': "0710d532e446af11a39ab1c8d3d4b5281b94cfd91550eb8f2cf6d9ff669b3290" # epm_api_v21 ask_proc_main
        },
        {
        'id_from': "2a9c63f0-5e9d-44a8-9c6e-92a5684030be", #  watsonx ask_proc_main
        'id_to': "299017c6-1add-44c3-a5dd-8f7cb2eecb8a" # watsonx ask_proc_main
        },
        {
        'id_from': "cc50493e455de183d661b95360196ddb468a02051add6dcaf4a6479b34900be5", #   watsonx ask_proc_main
        'id_to': "cc50493e455de183d661b95360196ddb468a02051add6dcaf4a6479b34900be5" # watsonx ask_proc_main
        }
]

In [ ]:
# Auhentification 
import json
from ibm_watson import AssistantV2
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

authenticator = IAMAuthenticator(API_KEY)
assistant = AssistantV2(
    version='2021-06-14',
    authenticator = authenticator
)

assistant.set_service_url(URL)



# Create folder for backup for data and logs 

In [ ]:
# get current datetime in string
import datetime as dt
date_name = dt.datetime.now().strftime("%Y-%m-%d_%H_%M_%S")
folder_name = date_name
BACKUP_FOLDER = BACKUP_FOLDER+folder_name
date_name

In [ ]:
os.makedirs(BACKUP_FOLDER)

In [ ]:
BACKUP_FOLDER

In [ ]:
logs_path = BACKUP_FOLDER+'/CHANGE_LOG_DESCRIPTION.txt'
with open(logs_path, mode='w') as f:
    f.write(CHANGE_LOG_DESCRIPTION)
    f.close()

# Changes setup 
In our case we only have 2 custom expensions and only one is the most commun to have changes so that is why I have to flows 
One when I only want to change the ids for EPM api And other when I want to change epm and watsonx.ai ids 


# Getting the assistant ID using the assistant name

In [ ]:
assistant_list = assistant.list_assistants()

In [ ]:
# find your assistant by name 


for bot in assistant_list.get_result().get('assistants'):
    if bot.get('name') == ASSISTANT_FROM:
        ASSISTANT_FROM_ID = bot.get('assistant_id')
        pprint.pprint(bot)
    if bot.get('name') == ASSISTANT_TO:
        ASSISTANT_TO_ID = bot.get('assistant_id')
        pprint.pprint(bot)

print(f'\n\nYour assistant FROM id is: {ASSISTANT_FROM_ID}')
print(f'\n\nYour assistant TO id is: {ASSISTANT_TO_ID}')

# Get the skills data from your assistant.

In [ ]:
'''
A successful call to this method only initiates an asynchronous export. 
The exported JSON data is not available until processing completes.

After the initial request is submitted, you can poll the status of the operation by calling the same request again and 
checking the value of the status property. If an error occurs (indicated by a status value of Failed), 
the status_description property provides more information about the error, and the status_errors property 
contains an array of error messages that caused the failure.

When processing has completed, the request returns the exported JSON data. Remember that the usual rate limits apply.
'''

skills_from = assistant.export_skills(assistant_id=ASSISTANT_FROM_ID)

In [ ]:
# here you get the json data 
assistand_from_data = skills_from.result
assistand_from_data

In [ ]:
assistand_from_data.keys()

In [ ]:
assistand_from_data.get('assistant_skills')[1].get('workspace').get('actions')[1].get('steps')[0]

# Transform the assistant_skills data into text 

In [ ]:
assistant_from_skills_txt = str(assistand_from_data['assistant_skills'])

In [ ]:

file_name = BACKUP_FOLDER+'/'+ASSISTANT_FROM+'_DOWNLOAD_'+date_name+'.txt'
print(file_name)

with open(file=file_name, mode='w') as f:
    f.write(assistant_from_skills_txt)
    f.close()

# Process to replace strings in assistant 

With the data in string format you can now use the replace method to replace the catalog and hash ids 

In [ ]:
for i in MODIFY:
    assistant_from_skills_txt = assistant_from_skills_txt.replace(i['id_from'], i['id_to'])

# After replacement 

Once the ids are replaced , now we can transform the text again to a list using eval function. 

In [ ]:
# saving the file as upload 
upload_name = BACKUP_FOLDER+'/'+ASSISTANT_FROM+'_UPLOAD_'+date_name+'.txt'
print(upload_name)

with open(file=upload_name, mode='w') as f:
    f.write(assistant_from_skills_txt)
    f.close()

In [ ]:
# then we can go back to list here.
assistant_from_skills_list = eval(assistant_from_skills_txt)

# Upload updated skills 

Now we can upload the skills data again in the server 

In [ ]:
import_skills = assistant.import_skills(assistant_id=ASSISTANT_TO_ID, 
                                        include_audit=True, 
                                        assistant_skills=assistant_from_skills_list, 
                                        assistant_state = assistand_from_data['assistant_state'])

# Check the import state 

In [ ]:
assistant.import_skills_status(assistant_id=ASSISTANT_TO_ID).get_result()